### Квантует DeepLabV3 MobilenetV3

Стартуем с трейнлупа, который нам выдали pytorch

Датасет COCO, https://cocodataset.org/#download 
Качаем train2017 и val2017

Можно использовать [сабсет](https://drive.google.com/file/d/1qdtAbK-iOsgJZxjbBva0pw2Vi5penjPc/view?usp=sharing) трейна на 20000, но тогда заранее залезте в класс датасета, и добавте работу с пропущенными картинками

Баллы: 20 баллов Static Quantization + 20 баллов Quantization Aware Training + 10 баллов Distillation

In [3]:
!pip3 install pycocotools

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 49.3 MB/s eta 0:00:00


In [1]:
import datetime
import os
import pickle
import time
from copy import deepcopy
from pathlib import Path

import torch
import torch.utils.data
from torch import nn
from torch.ao.quantization.quantize_fx import convert_fx
from torch.ao.quantization.quantize_fx import fuse_fx
from torch.optim.lr_scheduler import PolynomialLR
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from tqdm import tqdm

import utils
from quantization_utils.fake_quantization import fake_quantization
from quantization_utils.static_quantization import quantize_static
from speed_up_nn.hw_03.deeplab_quantization_ready.train import evaluate
from train import get_dataset
from train import train_one_epoch

In [2]:
# Вытащил дефолтные аргументы, чтобы не упражняться с argparse в ноутбуке
with Path('./torch_default_args.pickle').open('rb') as file:
    args = pickle.load(file)

In [3]:
# Подобирайте под ваше железо
args.data_path = '/data/coco/'
args.epochs = 1
args.batch_size = 16
args.workers = 16

In [4]:
args

Namespace(amp=False, aux_loss=False, backend='pil', batch_size=16, data_path='/data/coco/', dataset='coco', device='cuda', dist_url='env://', epochs=1, lr=0.01, lr_warmup_decay=0.01, lr_warmup_epochs=0, lr_warmup_method='linear', model='deeplabv3_mobilenet_v3_large', momentum=0.9, output_dir='.', print_freq=10, resume='', start_epoch=0, test_only=False, use_deterministic_algorithms=False, use_v2=False, weight_decay=0.0001, weights=None, weights_backbone=None, workers=16, world_size=1)

### Сначала просто валидация обычной сетки, прям на гпу

In [4]:
model = deeplabv3_resnet50(weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1)
model.eval();

In [5]:
if args.output_dir:
    utils.mkdir(args.output_dir)

utils.init_distributed_mode(args)

device = torch.device(args.device)

dataset_test, num_classes = get_dataset(args, is_train=False)

test_sampler = torch.utils.data.SequentialSampler(dataset_test)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=16, sampler=test_sampler, num_workers=args.workers, collate_fn=utils.collate_fn
)

Not using distributed mode
loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [6]:
import torch
from torch.utils.data import DataLoader, Subset, SequentialSampler
import numpy as np

def get_subset(dataset, ratio=0.5, batch_size=1, num_workers=16):
    dataset_size = len(dataset)
    subset_size = int(dataset_size * ratio)
    indices = list(map(int, np.random.permutation(dataset_size)[:subset_size]))
    subset = Subset(dataset, indices)
    sampler = SequentialSampler(subset)
    subset_loader = DataLoader(
        subset, batch_size=batch_size, sampler=sampler, 
        num_workers=num_workers, collate_fn=utils.collate_fn
    )

    return subset_loader


In [7]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
model.cuda()
subset = get_subset(dataset_test, ratio=0.2, batch_size=16)
confmat = evaluate(model, subset, device=device, num_classes=num_classes)
print(confmat)

evaluating...
Test:  [ 0/63]  eta: 0:06:25  batch_time: 3.7221 (3.7221)  time: 6.1194  data: 2.3972  max mem: 4874
Test:  [10/63]  eta: 0:02:09  batch_time: 2.0445 (2.1954)  time: 2.4421  data: 0.2467  max mem: 6720
Test:  [20/63]  eta: 0:01:46  batch_time: 2.0445 (2.3287)  time: 2.2948  data: 0.0357  max mem: 12013
Test:  [30/63]  eta: 0:01:16  batch_time: 1.8603 (2.1922)  time: 2.2272  data: 0.0367  max mem: 12013
Test:  [40/63]  eta: 0:00:50  batch_time: 1.7907 (2.1050)  time: 1.9014  data: 0.0313  max mem: 12013
Test:  [50/63]  eta: 0:00:27  batch_time: 1.7936 (2.0650)  time: 1.8979  data: 0.0300  max mem: 12013
Test:  [60/63]  eta: 0:00:06  batch_time: 1.7632 (2.0241)  time: 1.8878  data: 0.0294  max mem: 12013
Test: Total time: 0:02:10
Latency: 2.0073 sec/batch
Throughput: 7.64 samples/sec
global correct: 93.2
average row correct: ['96.4', '88.4', '80.2', '58.1', '52.1', '60.2', '78.9', '65.7', '90.1', '35.1', '88.7', '55.6', '84.4', '87.3', '86.2', '90.2', '50.5', '81.9', '52.6'

/home/hflabs/bot/DeepSchool/speed_up_nn/hw_03/deeplab_quantization_ready/utils.py:295: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(val)


### Заквантуем статические сетку, посмотрим на точность и скорость

In [8]:
# Квантуем
# Делаем fuse, делаем quantize_static и quantize_utils (посмотрите что там с кодом)
# Можно покрутить параметр num_batches, чтобы посмотреть сколько нужно данных на калибровку
# fused_model = fuse_fx(model) # KeyError: 'backbone.layer1.0.conv1.1'
q_model = quantize_static(
    model=model,
    data_loader=get_subset(dataset_test, ratio=0.1, batch_size=16),
    num_batches=16,
)

/home/hflabs/anaconda3/envs/ds/lib/python3.8/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [ ]:
# Замерим скорость квантованной модели на CPU
# Не забываем, от размера батча будет зависить буст

In [9]:
# Замеряем с бачтом 1, буста нет (3 batch/s)
subset_1batch = get_subset(dataset_test, ratio=0.1, batch_size=1)
confmat_1_batch = evaluate(q_model, subset_1batch, device='cpu', num_classes=num_classes)
print(confmat_1_batch)

# Замеряем с батчом 32, буст есть
subset_16batch = get_subset(dataset_test, ratio=0.1, batch_size=16)
confmat_16_batch = evaluate(q_model, subset_16batch, device='cpu', num_classes=num_classes)
print(confmat_16_batch)
# Мораль, latency != throughput. В сетке всегда есть накладные расходы, кроме перемалывания матричек

evaluating...
Test:  [  0/500]  eta: 0:08:04  batch_time: 0.4801 (0.4801)  time: 0.9691  data: 0.4888  max mem: 12013
Test:  [ 10/500]  eta: 0:03:05  batch_time: 0.3357 (0.3315)  time: 0.3778  data: 0.0452  max mem: 12013
Test:  [ 20/500]  eta: 0:02:51  batch_time: 0.3212 (0.3326)  time: 0.3274  data: 0.0010  max mem: 12013
Test:  [ 30/500]  eta: 0:02:42  batch_time: 0.3045 (0.3282)  time: 0.3283  data: 0.0007  max mem: 12013
Test:  [ 40/500]  eta: 0:02:36  batch_time: 0.3125 (0.3259)  time: 0.3206  data: 0.0005  max mem: 12013
Test:  [ 50/500]  eta: 0:02:35  batch_time: 0.3284 (0.3344)  time: 0.3462  data: 0.0009  max mem: 12013
Test:  [ 60/500]  eta: 0:02:29  batch_time: 0.3284 (0.3306)  time: 0.3425  data: 0.0009  max mem: 12013
Test:  [ 70/500]  eta: 0:02:26  batch_time: 0.3377 (0.3317)  time: 0.3267  data: 0.0009  max mem: 12013
Test:  [ 80/500]  eta: 0:02:22  batch_time: 0.3426 (0.3307)  time: 0.3329  data: 0.0009  max mem: 12013
Test:  [ 90/500]  eta: 0:02:17  batch_time: 0.3275

/home/hflabs/bot/DeepSchool/speed_up_nn/hw_03/deeplab_quantization_ready/utils.py:295: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(val)


Test:  [ 0/32]  eta: 0:09:49  batch_time: 14.9917 (14.9917)  time: 18.4106  data: 3.4188  max mem: 12013
Test:  [10/32]  eta: 0:04:23  batch_time: 10.6069 (11.6406)  time: 11.9851  data: 0.3113  max mem: 12013
Test:  [20/32]  eta: 0:02:11  batch_time: 10.1415 (10.7310)  time: 10.5540  data: 0.0007  max mem: 12013
Test:  [30/32]  eta: 0:00:21  batch_time: 9.4150 (10.5005)  time: 9.9081  data: 0.0006  max mem: 12013
Test: Total time: 0:05:32
Latency: 10.2317 sec/batch
Throughput: 1.51 samples/sec
global correct: 93.3
average row correct: ['96.6', '92.3', '86.4', '84.7', '15.3', '76.8', '87.5', '70.2', '82.7', '38.2', '87.1', '56.5', '69.6', '90.1', '85.3', '90.2', '64.4', '88.2', '71.2', '66.4', '60.8']
IoU: ['92.6', '83.7', '68.2', '75.2', '14.9', '60.7', '81.4', '53.2', '70.0', '31.0', '79.0', '37.6', '49.5', '80.2', '77.7', '81.9', '46.4', '78.9', '56.9', '62.6', '53.8']
mean IoU: 63.6


In [11]:
# Замерим скорость оригинальной модели на CPU
confmat_original_batch = evaluate(model.cpu(), subset_16batch, device='cpu', num_classes=num_classes)
print(confmat_original_batch)
# У меня на intel i9 при батчсайзе 32 получился x2 буст

evaluating...
Test:  [ 0/32]  eta: 0:27:39  batch_time: 48.6258 (48.6258)  time: 51.8678  data: 3.2419  max mem: 12013
Test:  [10/32]  eta: 0:14:55  batch_time: 36.4321 (40.3779)  time: 40.7074  data: 0.2954  max mem: 12013
Test:  [20/32]  eta: 0:07:28  batch_time: 34.8171 (37.1569)  time: 36.6203  data: 0.0007  max mem: 12013
Test:  [30/32]  eta: 0:01:12  batch_time: 32.4484 (36.3104)  time: 34.1087  data: 0.0006  max mem: 12013
Test: Total time: 0:18:56
Latency: 35.3861 sec/batch
Throughput: 0.44 samples/sec
global correct: 93.3
average row correct: ['96.3', '92.9', '86.6', '85.0', '15.5', '77.6', '88.0', '71.0', '82.0', '39.9', '86.5', '59.5', '70.5', '90.2', '86.7', '90.6', '67.0', '88.5', '71.6', '66.2', '61.1']
IoU: ['92.6', '82.8', '68.0', '75.9', '15.1', '60.1', '81.2', '53.3', '69.8', '31.7', '79.1', '38.4', '49.9', '79.7', '78.3', '82.0', '48.2', '78.5', '55.7', '61.5', '54.0']
mean IoU: 63.6


In [12]:
# Посчитаем метрики квантованной модели
# У меня была просадка где-то до 65 IoU
q_model.cpu()
confmat = evaluate(q_model, subset, device='cpu', num_classes=num_classes)
print(confmat)

evaluating...
Test:  [ 0/63]  eta: 0:14:31  batch_time: 10.5986 (10.5986)  time: 13.8304  data: 3.2316  max mem: 12013
Test:  [10/63]  eta: 0:09:07  batch_time: 10.2706 (10.0009)  time: 10.3261  data: 0.2941  max mem: 12013
Test:  [20/63]  eta: 0:08:08  batch_time: 10.2907 (11.1660)  time: 11.2325  data: 0.0008  max mem: 12013
Test:  [30/63]  eta: 0:05:56  batch_time: 9.5196 (10.6614)  time: 11.0645  data: 0.0007  max mem: 12013
Test:  [40/63]  eta: 0:03:59  batch_time: 8.9075 (10.3123)  time: 9.4501  data: 0.0005  max mem: 12013
Test:  [50/63]  eta: 0:02:13  batch_time: 9.0070 (10.1584)  time: 9.4104  data: 0.0009  max mem: 12013
Test:  [60/63]  eta: 0:00:30  batch_time: 8.6819 (9.9656)  time: 9.2860  data: 0.0009  max mem: 12013
Test: Total time: 0:10:28
Latency: 9.8939 sec/batch
Throughput: 1.59 samples/sec
global correct: 93.3
average row correct: ['96.6', '87.6', '79.1', '57.6', '51.2', '58.3', '78.5', '63.7', '90.6', '33.9', '88.5', '53.0', '83.1', '87.3', '84.9', '89.8', '48.4',

### Делаем Quantization Aware Training. Используем готовый трейнплуп от pytorch

In [ ]:
# Делаем фейк квантование, идём смотреть quantization_utils

### Тут берём из train.py скрипт main() и вытаскиваем трейн луп

1. Не забыть провалидировать модель fake quant до qat
2. Не забыть провалидировать модель после обучения
3. Конвертировать модель из fake quant в обычный quant
4. Проверить точность и скорость модели
Должно хватить пары эпох, lr надо будет покрутить.
Цель минимум это поднять точность конвертированной QAT модели (будет у вас QAT до обучения 55, станет 56, молодцы!)

In [ ]:
# Подсказка, учить надо в train и на GPU
qat_model.cuda();

In [ ]:
# Инференс делаем на cpu, предварительно конвертируя модельку на CPU
qat_model.cpu()
int_qat_model = convert_fx(qat_model)

In [ ]:
# Точность модели fake quant и квантованной после конвертации будут разные
# Так и должно быть, всё таки мы эмулировали квантование.

### Балуемся с дистилляцией
Врываемся в train.py и добавляем туда дистилляцию, просто по последнему слою (до софтмакса, на логитах) делаем стягивание по MSE

Цель поднять точность и ускорить сходимость.

Балуемся с весами обычного и distill лосса.

Можно вообще выкинуть classification loss и смоделировать ситуацию когда вам не выдали лейблов (жиза)